# Time travel (时间旅行)


In [1]:
from typing import Annotated
from dotenv import load_dotenv

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()


True

In [2]:
import os
from langchain_openai import ChatOpenAI

# from langchain.chat_models import init_chat_model
# llm = init_chat_model('deepseek:deepseek-chat')
llm = ChatOpenAI(
  model='Qwen/Qwen3-30B-A3B-Instruct-2507',
  openai_api_key=os.environ['SILICONFLOW_API_KEY'],
  openai_api_base='https://api.siliconflow.cn/v1',
  streaming=True,
)


class State(TypedDict):
  messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
  return {'messages': [llm_with_tools.invoke(state['messages'])]}


graph_builder.add_node('chatbot', chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node('tools', tool_node)

graph_builder.add_conditional_edges(
  'chatbot',
  tools_condition,
)
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')
graph_builder.add_edge('chatbot', END)

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

## Add steps

向图表中添加步骤。每个步骤将在其状态历史记录中进行检查点记录：


In [3]:
config = {'configurable': {'thread_id': '1'}}
events = graph.stream(
  {
    'messages': [
      {
        'role': 'user',
        # 若使用 deepseek，需要提示可以调用工具，不然模型自主调用工具的意愿很低
        'content': ('我正在学习 LangGraph。你能帮我做一些研究吗？你可以调用 TavilySearch 工具。'),
      },
    ],
  },
  config,
  stream_mode='values',
)
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

我正在学习 LangGraph。你能帮我做一些研究吗？你可以调用 TavilySearch 工具。
================================== Ai Message ==================================

当然可以！请告诉我您想了解 LangGraph 的哪些方面？例如：

- LangGraph 的基本概念和核心功能？
- 如何使用 LangGraph 构建复杂的应用程序？
- LangGraph 与其他框架（如 LangChain）的比较？
- LangGraph 的最新发展或社区动态？

告诉我您的具体需求，我会为您进行研究！


In [4]:
events = graph.stream(
  {
    'messages': [
      {
        'role': 'user',
        'content': ('是的，这很有帮助。也许我可以用它来构建一个自主智能体！你可以调用 TavilySearch 工具。'),
      },
    ],
  },
  config,
  stream_mode='values',
)
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

是的，这很有帮助。也许我可以用它来构建一个自主智能体！你可以调用 TavilySearch 工具。
================================== Ai Message ==================================
Tool Calls:
  tavily_search (0198971788786992f613965cdb919f7a)
 Call ID: 0198971788786992f613965cdb919f7a
  Args:
    query: LangGraph autonomous agent development tutorial
    search_depth: advanced
    topic: general
    include_images: False
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph autonomous agent development tutorial", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.youtube.com/watch?v=1w5cCXlh7JQ", "title": "LangGraph Tutorial - How to Build Advanced AI Agent ...", "content": "In this video, you're going to learn how to build AI agents using LangGraph. LangGraph is a much more professional and in-depth framework compared to something like LangChai

## Replay the full state history

现在已经向聊天机器人添加了步骤，可以 `replay` 查看完整的状态历史记录，以查看所有发生的内容。


In [5]:
to_replay = None
for state in graph.get_state_history(config):
  print('消息数量: ', len(state.values['messages']), '下一个节点: ', state.next)
  print('-' * 80)
  if len(state.values['messages']) == 6:
    # 我们根据消息数量选择一个特定的状态。
    to_replay = state

消息数量:  6 下一个节点:  ()
--------------------------------------------------------------------------------
消息数量:  5 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  4 下一个节点:  ('tools',)
--------------------------------------------------------------------------------
消息数量:  3 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  2 下一个节点:  ('__start__',)
--------------------------------------------------------------------------------
消息数量:  2 下一个节点:  ()
--------------------------------------------------------------------------------
消息数量:  1 下一个节点:  ('chatbot',)
--------------------------------------------------------------------------------
消息数量:  0 下一个节点:  ('__start__',)
--------------------------------------------------------------------------------


检查点会在图的每个步骤中保存。这跨越了方法调用，因此您可以回溯到整个线程的历史记录。

### Resume from a checkpoint


In [6]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f076603-4697-6060-8006-e1c60afe516b'}}


## Load a state from a moment-in-time

检查点的 `to_replay.config` 包含一个 `checkpoint_id` 时间戳。提供此 `checkpoint_id` 值会告知 LangGraph 的检查点工具从该时间点加载状态。


In [7]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
# `to_replay.config` 中的 `checkpoint_id` 对应于我们已保存到检查点工具的某个状态。
for event in graph.stream(None, to_replay.config, stream_mode='values'):
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================== Ai Message ==================================

太好了！您想构建一个自主智能体，而 LangGraph 正是实现这一目标的绝佳工具。以下是基于搜索结果的详细分析和建议：

---

### 🎯 **为什么 LangGraph 适合构建自主智能体？**

1. **强大的流程控制能力**  
   LangGraph 不只是简单地调用 LLM，而是允许你设计**复杂的、可执行的流程图**（Graph）。这意味着您可以定义：
   - 多阶段任务分解（如：规划 → 执行 → 检查 → 再规划）
   - 条件分支（如：“如果用户提出疑问，就切换到客服模式”）
   - 循环逻辑（如：自动重试失败的步骤）

2. **状态管理与记忆机制**  
   智能体需要“记住”上下文。LangGraph 提供了**可持久化状态**（State）系统，能追踪对话历史、任务进度、用户偏好等，真正实现“连续智能体行为”。

3. **可扩展的组件设计**  
   可以轻松集成：
   - 工具调用（如查询天气、查数据库）
   - 外部 API
   - 用户输入/输出接口
   - 权限控制与安全性

4. **可视化调试支持**  
   您可以通过 `generate_graph` 生成流程图，清晰看到智能体的决策路径，便于调试和优化。

---

### 🛠️ **构建你的第一个自主智能体：快速上手步骤**

根据 [Medium 上的实战教程](https://medium.com/@lorevanoudenhove/how-to-build-ai-agents-with-langgraph-a-step-by-step-guide-5d84d9c7e832)，这里是一条清晰路径：

#### ✅ 第一步：环境准备
```bash
pip install langgraph
```

#### ✅ 第二步：定义状态结构（State）
```python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END

class AgentState(Typ

图表从 tools 节点继续执行。您可以通过第一个打印的值是搜索引擎工具的响应来判断这一点。
